In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

import os
import numpy as np
import cv2
from tqdm.notebook import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [3]:
image_transforms = transforms.Compose([transforms.Resize((224,224)), 
                                       transforms.ToTensor(), 
                                       transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [4]:
data = {'train': ImageFolder(root = './train', transform = image_transforms),
        'test' : ImageFolder(root = './test', transform = image_transforms)}

In [5]:
data['train'].class_to_idx, data['test'].class_to_idx, len(data['train']), len(data['test'])

({'with_incorrect': 0, 'with_mask': 1, 'without_mask': 2},
 {'with_incorrect': 0, 'with_mask': 1, 'without_mask': 2},
 6126,
 1018)

In [6]:
dataloaders = {'train': DataLoader(data['train'], batch_size = 100, shuffle = True),
               'test' : DataLoader(data['test'], batch_size = 100, shuffle = True)}

In [7]:
model_res18 = models.resnet18(pretrained=True)
model_res18.fc = nn.Linear(model_res18.fc.in_features, 3)
model_res18.to(device)

In [9]:
optimizer = optim.Adam(model_res18.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()
train_loss = []
val_loss = []

In [10]:
epochs = 5
for epoch in tqdm(range(1,epochs+1)):
    for image, label in dataloaders['train']:
        image, label = image.to(device), label.to(device)

        model_res18.zero_grad()
        prediction = model_res18(image)
        loss = criterion(prediction, label)
        loss.backward()
        optimizer.step()
    print("EPOCH", epoch)
    print('training_loss: ', loss)
    train_loss.append(loss)

  0%|          | 0/5 [00:00<?, ?it/s]

EPOCH 1
training_loss:  tensor(0.2816, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH 2
training_loss:  tensor(0.1519, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH 3
training_loss:  tensor(0.0262, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH 4
training_loss:  tensor(0.2453, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH 5
training_loss:  tensor(0.0428, device='cuda:0', grad_fn=<NllLossBackward>)


In [11]:
correct = 0
total = 0
model_res18.eval()

with torch.no_grad():
    for image, label in dataloaders['test']:
        image, label = image.to(device), label.to(device)

        prediction = model_res18(image)
        #loss = loss_fun(prediction, label)
        #print(prediction)
        for k in range(len(prediction)):
            #print(k)
            #print(prediction)
            #print(label)
            if torch.argmax(prediction[k]) == label[k]:
                correct+=1
                #print(image.shape)
            total += 1

#validation_loss.append(loss)    
#print('validation_loss: ', loss)
print('accuracy: ', round(correct/total, 3))
print('correct ', correct)
print('total', total)

accuracy:  0.906
correct  922
total 1018


In [ ]:
'''model.eval() is a kind of switch for some specific layers/parts of the model that behave differently during training and inference (evaluating) time. For example, Dropouts Layers, BatchNorm Layers etc. You need to turn off them during model evaluation, and .eval() will do it for you. In addition, the common practice for evaluating/validation is using torch.no_grad() in pair with model.eval() to turn off gradients computation:

# evaluate model:
model.eval()

with torch.no_grad():
    ...
    out_data = model(data)
    ...
BUT, don't forget to turn back to training mode after eval step:

# training step
...
model.train()
...
'''